In [2]:
# 			                     Throughput			
# Host 	Guest 	mode 	Without TPT 	With TPT 		
# 4K	4K	     rnd 	158.685	        302.41		
# 2M	4K 	     rnd 	243.09	        274.843		
# 2M 	2M 	     rnd    372.582         378.718	  

           	
# 4K 	4K 	     seq 	8689.182	    9191.347		
# 2M 	4K 	     seq 	8926.269	    8997.371	
# 2M 	2M	     seq 	9213.024	    9668.146		

import numpy as np
import matplotlib.pyplot as plt

RANDOM_NATIVE = 328.203
SEQUENTIAL_NATIVE = 9454.887

def plot_throughput(data_without, data_with, mode, group_labels, filename, fmt='.1f'):
    n_groups = len(group_labels)
    index = np.arange(n_groups)
    bar_width = 0.35

    plt.figure(figsize=(8, 5))
    bars1 = plt.bar(index - bar_width/2, data_without, bar_width, label='Without TPT', color='#1f77b4')
    bars2 = plt.bar(index + bar_width/2, data_with, bar_width, label='With TPT', color='#ff7f0e')

    plt.xlabel('Host/Guest Configuration')
    plt.ylabel('Throughput')
    plt.title(f'{mode} Mode Throughput')
    plt.xticks(index, group_labels)
    plt.legend()

    # Function to add labels with specified format
    def add_labels(bars):
        for bar in bars:
            height = bar.get_height()
            plt.annotate(f'{height:{fmt}}',
                        xy=(bar.get_x() + bar.get_width() / 2, height),
                        xytext=(0, 3),
                        textcoords="offset points",
                        ha='center', va='bottom', fontsize=8)

    add_labels(bars1)
    add_labels(bars2)

    plt.tight_layout()
    plt.savefig(filename)
    plt.close()

# Data for random mode (values in IOPS)
random_without = [158.685, 243.09, 372.582]
random_with = [302.41, 274.843, 378.718]
group_labels = ['4K/4K', '2M/4K', '2M/2M']
normilized_random_without = [RANDOM_NATIVE / x for x in random_without]
normilized_random_with = [RANDOM_NATIVE / x for x in random_with]
plot_throughput(normilized_random_without, normilized_random_with, 'Random', group_labels, 'random_throughput.png', fmt='.3f')

# Data for sequential mode (values in IOPS)
sequential_without = [8689.182, 8926.269, 9213.024]
sequential_with = [9191.347, 8997.371, 9668.146]
normilized_sequential_without = [SEQUENTIAL_NATIVE / x for x in sequential_without]
normilized_sequential_with = [SEQUENTIAL_NATIVE / x for x in sequential_with]
plot_throughput(normilized_sequential_without, normilized_sequential_with, 'Sequential', group_labels, 'sequential_throughput.png', fmt='.3f')



